In [1]:
import os
import json

import boto3
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role, Session



sess = Session()

role = get_execution_role()
model_dir = "/home/ec2-user/SageMaker/smoke-detection-smokeynet/src" # Replace with S3

In [2]:
pt_mnist_model_data = sess.upload_data(
    path="/home/ec2-user/SageMaker/smoke-detection-smokeynet/src/last.tar.gz", bucket=sess.default_bucket(), key_prefix="model/pytorch"
)

In [3]:
model = PyTorchModel(
    entry_point="inference.py",
    source_dir=model_dir,
    role=role,
    model_data=pt_mnist_model_data,
    framework_version="1.5.0",
    py_version="py3",
)

#### Deploy Model

In [4]:
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer


local_mode = False

if local_mode:
    instance_type = "local"
else:
    instance_type = "ml.g4dn.xlarge"

predictor = model.deploy(
    initial_instance_count=1,
    instance_type=instance_type,
    serializer=JSONSerializer(),
    deserializer=JSONDeserializer(),
)

----------!

In [5]:
import random
import numpy as np
import boto3
from io import BytesIO
import pickle
from urllib.parse import urlparse
#dummy data must be replaced with processed image tiles
dummy_data = np.random.rand(2, 45, 2, 3, 224, 224)
s3_client = boto3.client("s3")
processed_data = BytesIO()
pickle.dump(dummy_data, processed_data)
processed_data.seek(0)
s3_client.upload_fileobj(processed_data, 'smokynet-inference-images-processed', 'processed_data.pkl')

In [6]:


res = predictor.predict({'bucket_name' : 'smokynet-inference-images-processed','file_name':'processed_data.pkl'})

In [ ]:
!pip install transformers

#### Delete endpoint

In [ ]:
import os

if not local_mode:
    predictor.delete_endpoint()
else:
    os.system("docker container ls | grep 8080 | awk '{print $1}' | xargs docker container rm -f")